In [1]:
!pip install ultralytics supervision deep_sort_realtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

**Import Libraries**

In [2]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from IPython.display import HTML
from base64 import b64encode

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Saving 20250510_0639_Graceful Movements in Harmony_simple_compose_01jtxa5by4e6b8621rwr39rp7h.mp4 to 20250510_0639_Graceful Movements in Harmony_simple_compose_01jtxa5by4e6b8621rwr39rp7h.mp4




*   The YOLO 10  model is loaded with the pre-trained weights for object detection.


*   DeepSort is initialized to track objects across frames, with a maximum age of 30 frames before losing track of an object.




In [4]:
model = YOLO('yolov10n.pt')

100%|██████████| 5.59M/5.59M [00:00<00:00, 110MB/s]


In [5]:
tracker = DeepSort(max_age=30)



*   track_history is used to store the paths of tracked objects.



*   colors is a list of colors to use for drawing the paths and bounding boxes of the detected persons.



In [6]:
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
save_path = "/content/test1.avi"
out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))



*   track_history is used to store the paths of tracked objects.

*   colors is a list of colors to use for drawing the paths and bounding boxes of the detected persons.



In [7]:
track_history = {}
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

In [8]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
#YOLO is used to detect objects in the current frame, specifically looking for "person" class.
    results = model.predict(source=frame, conf=0.5, classes=[0], verbose=False)[0]
    detections = []
#For each detected person, we extract the bounding box coordinates and score, and prepare them for tracking.
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = box
        detections.append(([x1, y1, x2 - x1, y2 - y1], score, 'person'))
#The DeepSort tracker is updated with the current frame's detections.
    tracks = tracker.update_tracks(detections, frame=frame)

#For each confirmed track, calculate the center of the bounding box and update the track's path history.
    for track in tracks:
        if not track.is_confirmed():
            continue
#The track_id ensures that each person is consistently identified across frames, even if they cross paths.
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        color = colors[int(track_id) % len(colors)]

        if track_id not in track_history:
            track_history[track_id] = []

        track_history[track_id].append((cx, cy))

#colored line is drawn for each person, and their bounding box and track ID are displayed on the frame.
        for j in range(1, len(track_history[track_id])):
            cv2.line(frame, track_history[track_id][j - 1], track_history[track_id][j], color, 2)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'Person {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()

In [9]:
compressed_path = "/content/result_compressed.mp4"

In [10]:
output_path = save_path
!ffmpeg -i {output_path} -vcodec libx264 -crf 28 /content/output_final.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
with open("/content/output_final.mp4", "rb") as f:
    mp4 = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display(HTML(f"""
<video width=600 controls>
  <source src="{data_url}" type="video/mp4">
</video>
"""))

In [ ]:
compressed_path = "/content/result_compressed.mp4"
from google.colab import files
files.download(compressed_path)